# LOAD DATA FROM ESHKOL AUGUST 2020

Eshkol said:

The matrices named fncd are 4D: 128x128x100x33, where horizontal resolution is 50 m and vertical is 40 m. 

The 33 are the size distribution bins according to the same rd.mat vector that I gave you in all previous files.

Number concentration = sum( fncd ) on the forth dimension 

effective radius - re(i,j,k) = sum( rd^3*fncd(i,j,k) )/ sum( rd^2*fncd(i,j,k) )

So from the nc files (each contains x,y,z,time,rn,rd,p,rho,FNCD) I need to extract:

- 'FNCD': <class 'netCDF4._netCDF4.Variable'>

- 'rd': <class 'netCDF4._netCDF4.Variable'>

Each nc file has:

[variable units shape]

- x m (128,)

- y m (128,)

- z m (100,)

- time d (1,)

- rn micron (33,)

- rd micron (33,)

- p mb (100,)

- r ho g/m3 (100,)

- FNCD #/cm3/bin  (1, 33, 100, 128, 128)

-----

To work with nc files you need netcdf4 package. 

So in the terminal do:

`conda install -c anaconda netcdf4`

then use: `from netCDF4 import Dataset`

-----

What is FNCD?

I tried to read:

https://en.wikipedia.org/wiki/Raindrop_size_distribution

# Imports

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches
import glob
import os
import pickle
import scipy
import math 
import scipy.io as sio

from scipy.optimize import curve_fit
import re
from tqdm import tqdm
from netCDF4 import Dataset

# Helper Functions

In [46]:
def gamma_distribution(r,N,re,ve):
    C = ((re*ve)**(2-(1/ve)))/math.gamma((1/ve)-2)
    t = N*C*(r**((1/ve)-3))
    n = t*np.exp(-r/(re*ve))
    return n

def gamma_distribution_tofit(r,N,alpha,b):
    # defination from: https://web.archive.org/web/20171213043637/http://nit.colorado.edu/shdom/shdomdoc/
    a = (b**(alpha+1))*(N/(math.gamma(alpha+1)))#N = a Gamma(alpha+1)/ b^(alpha+1)
    n = a*(r**alpha)*np.exp(-r*b)
    return n

In [47]:
# these parameters should be known in advence.
dx,dy,dz=(1e-3*50,1e-3*50,1e-3*40) # in km
nz = 100
z_min = 0
z_max = 1e-3*40*nz
zgrid = np.linspace(z_min, z_max-dz ,nz)

In [48]:
data_dir = '/wdata/clouds_from_weizmann/BOMEX_512X512X170_500CCN_20m' # data should be stored under this path 
# format_ = 'BOMEX_128x128x100_2000CCN_50m_micro_128_*.com3D_int_2.nc'# lod one was .com3D_int_2.nc
format_ = 'BOMEX_512x512x170_500CCN_20m_micro_256_*.com3D_int_2.nc' # the format of the files to search for.
volumes_paths = sorted(glob.glob(data_dir + '/'+format_))
volume_stamps = [re.findall('_(\d*).com3D_int_2.nc', i)[0] for i in volumes_paths]
volume_stamps = [int(i) for i in volume_stamps]# convert to integer 
print(f'Number of sections in {data_dir} of format {format_} is {len(volumes_paths)}')

Number of sections in /wdata/clouds_from_weizmann/BOMEX_512X512X170_500CCN_20m of format BOMEX_512x512x170_500CCN_20m_micro_256_*.com3D_int_2.nc is 301


In [49]:
SHOWVOL = True # if true, show evrey volume
IF_SAVE_txt = True
PRINT_MICRO_PHYSICS = False
FIT_GAMMA = False # if it True, then fit gamma parameters with non-linear least squares fit
# BUT, the fit works strange, it doesn't fit good parameters to all vaxels. Some get huge values.

Example File loading and params

In [50]:
file = volumes_paths[0]
print('loading ',file)
nc = Dataset(file)
nc

loading  /wdata/clouds_from_weizmann/BOMEX_512X512X170_500CCN_20m/BOMEX_512x512x170_500CCN_20m_micro_256_0000021600.com3D_int_2.nc


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    dimensions(sizes): x(512), y(512), z(170), time(1), rn(33), rd(33), ib(33)
    variables(dimensions): float32 x(x), float32 y(y), float32 z(z), float32 time(time), float32 rn(rn), float32 rd(rd), float32 p(z), float32 rho(z), float32 FNCD(time,rd,z,y,x)
    groups: 

In [51]:
# NOT USED
def get_dx_dy(nc):
    """
        Extracts dx, dy from .nc file
    """
    x, y = nc.variables['x'][:].data, nc.variables['y'][:].data
    dx, dy = np.unique(np.diff(x)), np.unique(np.diff(y)) 
    if len(dx) != 1:
            raise(f'dx is not uniform. dx: {np.diff(x)}')
    if len(dy) != 1:
            raise(f'dy is not uniform. dy: {np.diff(y)}')
    
    return dx, dy

In [52]:
for file in volumes_paths:
    print('loading ',file)
    nc = Dataset(file)
    
    rd = nc.variables['rd'][:].data # units micron
    FNCD = np.squeeze(nc.variables['FNCD'][:].data) # units #/cm3/bin
    # note that the shape of FNCD is now (33, 100, 128, 128)
    
    # LWC:
    # what Eshkol explained with old data: rho - density kg/m^3  - you can use this to convert mixing ratio (LWC [g/kg]]) to liquid water density [g/m^-3]]
    # rho - density g/m3:
    rho = 1e6
    # Eshkol said: The rho in the output is of the whole voxel (so its mostly density of dry air and water vapor. 
    # I don't think you need it. For liquid water content you can take water density as constant of  1 g/cm^3.
    # rho = nc.variables['rho'][:].data
    
    # ----------------------------------------------------------------------------------------
    # ----------------------------------------------------------------------------------------
    # ----------------------------------------------------------------------------------------
    print('performing operations(?)...')
    DOPC = FNCD# distribution of droplets consintration, it is given as histogram
    e_DOPC = DOPC
    e_rd = rd[:,np.newaxis, np.newaxis, np.newaxis]
    # ---------------------------------------------
    Dx = np.diff(rd)[:, np.newaxis, np.newaxis, np.newaxis]
    DOPC = DOPC[:-1,...]/Dx # it is the PDF
    rd = rd[:-1, np.newaxis, np.newaxis, np.newaxis]

    #---------------------------------------------
    # calculate reff, veff, LWC from the FNCD (here it doesn't assume gamma distribution):
    reff3D = np.trapz(DOPC*rd**3,rd, axis=0)/np.trapz(DOPC*rd**2,rd, axis=0)
    veff3D = np.trapz(((rd-reff3D)**2)*(rd**2)*DOPC,rd, axis=0)/(reff3D**2*np.trapz(DOPC*rd**2,rd, axis=0))
    LWC3D = (1e-12)*(4/3)*np.pi*rho*np.trapz((rd**3)*DOPC,rd, axis=0)
    # above it uses integral of the PDF, below it is as Eshkol suggested, just to use sum:
    # I think the integral is better choice espesialy using np.trapz, but I will use the sum as Yoav told to doas Eshkol instructed.
    e_reff3D = np.sum( e_rd**3*e_DOPC , axis=0)/ np.sum( e_rd**2*e_DOPC , axis=0)
    e_veff3D = np.sum( ((e_rd-e_reff3D)**2)*(e_rd**2)*e_DOPC , axis=0)/ np.sum( e_reff3D**2*e_DOPC*e_rd**2 , axis=0)
    e_LWC3D = (1e-12)*(4/3)*np.pi*rho*np.sum((e_rd**3)*e_DOPC, axis=0)
    # units of rho are g/m^3, of DOPC are #/cm3/um, of r um. The units of the result LWC are [g/m^3]
    NC3D = np.sum(DOPC*Dx, axis=0)# total consintration

    # avoide nans:
    for arr in [e_LWC3D, e_reff3D, e_veff3D, LWC3D, reff3D, veff3D]:
        arr = np.nan_to_num(arr)

    # here the 3d shape is (100, 128, 128)
    RE = np.transpose(e_reff3D, (2, 1, 0))
    VE = np.transpose(e_veff3D, (2, 1, 0))
    LWC = np.transpose(e_LWC3D, (2, 1, 0))
    NCT = np.transpose(NC3D, (2, 1, 0))
    # here the 3d shape is (128, 128,100)
    
    # this is a good place to filter voxels.
    # Eshkol said voxels with LWC < 0.01 can be considered as not cloudy ones.
    lwc_treshold = 0.01 # [g/m^3]
    # In addition, I want to filter voxels with NC<1.
    NC_treshold = 1
    for arr in [RE, VE, NCT, LWC]:
        arr[LWC<lwc_treshold]  = 0

    for arr in [RE, VE, LWC, NCT]:
        arr[NCT<NC_treshold]  = 0        
    
    # re treshold:
#     re_treshold = 35
#     VE[RE>re_treshold]  = 0
#     LWC[RE>re_treshold] = 0
#     NCT[RE>re_treshold] = 0
#     RE[RE>re_treshold]  = 0
    
    
    
    # fit gamma:
    if(FIT_GAMMA):
        
        indxs = np.argwhere(LWC > 0)# cloudy voxel indexes
        for vox_indx in tqdm(indxs):
            DOPC_voxel = DOPC[:,vox_indx[2],vox_indx[1],vox_indx[0]] # distribution of droplets consintration 
            # the order vox_indx[2],vox_indx[1],vox_indx[0] is because x = np.transpose(x, (2, 1, 0))          
            try:
                PRINT_MICRO_PHYSICS = False
                rd_ = np.squeeze(rd)
                popt, pcov = curve_fit(gamma_distribution_tofit, rd_, DOPC_voxel)
                fited_gamma = gamma_distribution_tofit(rd_, *popt)
                fit_reff = np.trapz(fited_gamma*rd_**3,rd_, axis=0)/np.trapz(fited_gamma*rd_**2,rd_, axis=0)
                fit_veff = np.trapz(((rd_-fit_reff)**2)*(rd_**2)*fited_gamma,rd_, axis=0)/(fit_reff**2*np.trapz(fited_gamma*rd_**2,rd_, axis=0))
                fit_LWC = (1e-12)*(4/3)*np.pi*rho*np.trapz((rd_**3)*fited_gamma,rd_, axis=0)
                if(np.isnan(fit_reff) or np.isnan(fit_veff) or np.isnan(fit_LWC)):
                    continue
                
                if(fit_reff>re_treshold or fit_LWC<lwc_treshold or popt[0] < NC_treshold):
                    continue
                    
                if(PRINT_MICRO_PHYSICS):
                    print("fited LWC = {} [g/m^3]".format(fit_LWC))
                    print("fited reff = {} [um]".format(fit_reff))
                    print("fited veff = {}\n".format(fit_veff))
                    
                RE[vox_indx[0],vox_indx[1],vox_indx[2]]  = fit_reff
                VE[vox_indx[0],vox_indx[1],vox_indx[2]]  = fit_veff
                LWC[vox_indx[0],vox_indx[1],vox_indx[2]] = fit_LWC 
            except:
                print("Optimal parameters not found.\It will uses original parameters:")
                print("calculated LWC = {} [g/m^3]".format(LWC[vox_indx[0],vox_indx[1],vox_indx[2]]))
                print("calculated reff = {} [um]".format(RE[vox_indx[0],vox_indx[1],vox_indx[2]]))
                print("calculated veff = {}".format(VE[vox_indx[0],vox_indx[1],vox_indx[2]]))
    
    
    # do the padding here. Pad with zeros on the sides
    npad = ((1,1),(1,1),(0,0))
    for arr in [LWC, NCT, RE, VE]:
        arr = np.pad(arr, pad_width=npad, mode='constant', constant_values=0.0)
    
    if(IF_SAVE_txt):
        comment_line = "Data from Eshkol Itan"
        original_name = os.path.basename(file)
        file_name = original_name.split('.')[0]+'.txt'
        file_name = os.path.join(data_dir,file_name)
        print('saving {}'.format(file_name))
        
        # print RE min max and VE nim max:
        m=RE[RE>0]
        print(f"RE MIN = {m.min()}")
        print(f"RE MAX = {RE.max()}")

        m=VE[VE>0]
        print(f"VE MIN = {m.min()}")
        print(f"VE MAX = {VE.max()}")
        # --------finish to print RE min max and VE nim max:

        np.savetxt(file_name, X=np.array([RE.shape]), fmt='%d', header=comment_line)

        with open(file_name, 'ab') as f:
            np.savetxt(f, X=np.concatenate((np.array([dx, dy]), zgrid)).reshape(1,-1), fmt='%2.3f')
            nx, ny, nz = RE.shape
            totext_lwc = LWC
            totext_reff = RE
            totext_veff = VE
            y, x, z = np.meshgrid(range(ny), range(nx), range(nz))
            data = np.vstack((x.ravel(), y.ravel(), z.ravel(), totext_lwc.ravel(), totext_reff.ravel(), totext_veff.ravel())).T
            np.savetxt(f, X=data, fmt='%d %d %d %.5f %.3f %.5f')        
        
        # save only RE,VE(+LWC) for visualization in matlab:
        file_name = original_name.split('.')[0]+'_ONLY_RE_VE_LWC.mat'
        file_name = os.path.join(data_dir,file_name)
        
        x, y, z = nc.variables['x'][:].data, nc.variables['y'][:].data, nc.variables['z'][:].data

        sio.savemat(file_name, dict(reff=RE,veff=VE,lwc=LWC, x=x,y=y,z=z))  
        print(f"saving reff, veff , lwc .mat file to: {file_name}\n\n")
        
    break
            
# in pyshdom liquid water content lwc is of units (g/m^3).

loading  /wdata/clouds_from_weizmann/BOMEX_512X512X170_500CCN_20m/BOMEX_512x512x170_500CCN_20m_micro_256_0000021600.com3D_int_2.nc
performing operations(?)...


/home/shubi/miniconda3/envs/pyshdom/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/home/shubi/miniconda3/envs/pyshdom/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


saving /wdata/clouds_from_weizmann/BOMEX_512X512X170_500CCN_20m/BOMEX_512x512x170_500CCN_20m_micro_256_0000021600.txt
RE MIN = 2.2485601902008057
RE MAX = 17.531906127929688
VE MIN = 0.00581858167424798
VE MAX = 0.5857473015785217
saving reff, veff , lwc .mat file to: /wdata/clouds_from_weizmann/BOMEX_512X512X170_500CCN_20m/BOMEX_512x512x170_500CCN_20m_micro_256_0000021600_ONLY_RE_VE_LWC.mat


